In [1]:
import json
import gensim
import nltk


Using Theano backend.


In [2]:
# !apt-get install libicu-dev

In [3]:
# !pip3 install pyicu
#!pip3 install pycld2
# !pip3 install morfessor

In [4]:
import polyglot

In [5]:
from polyglot.text import Text as T

In [6]:
from polyglot.downloader import downloader
# downloader.download("sentiment2.ru")

In [7]:
# downloader.download("ner2.ru")

In [8]:
# downloader.download("embeddings2.ru")

In [9]:
# downloader.download("morph2.ru")

In [10]:
from collections import defaultdict

In [11]:
# nltk.download('punkt')

In [13]:
import re

In [14]:
from nltk.tokenize import sent_tokenize, word_tokenize

In [15]:
stemmer = nltk.stem.snowball.RussianStemmer()

In [16]:
def word_tokenize_filt(sent, do_lowercase=False, do_stem=False):
    words = []
    for word in word_tokenize(sent):
        word = re.sub('[\"\«\»]', "", word)
        if not word:
            continue
        if do_lowercase:
            word = word.lower()
        if do_stem:
            word = stemmer.stem(word)
        words.append(word)
    return words

In [17]:
def preprocess_data(data, do_lowercase=False, do_stem=False):
    for i in range(len(data)):
        text = data[i]["text"]
        text = re.sub("<[^>]*>", "", text) # remove tags
        data[i]["processed_text_sents"] = [word_tokenize_filt(sent, do_lowercase, do_stem)
                                         for sent in sent_tokenize(data[i]["text"])]
        data[i]["processed_text"] = word_tokenize_filt(data[i]["text"], True, True)
        text = T(text, hint_language_code="ru")
        try:
            data[i]["polarity"] = text.polarity
            data[i]["entities"] = [(e.tag, e[0]) for e in text.entities]
        except:
            data[i]["polarity"] = 0
            data[i]["entities"] = []
    return data


In [18]:
import os

In [19]:
for filename in os.listdir("texts/"):
    print(filename)
    with open("texts/%s" % filename, "r") as inpf:
        try:
            data = json.load(inpf)
            data = preprocess_data(data)
        except BaseException as e:
            print(e)
    with open("processed_texts/%s" % filename, "w") as outf:
        json.dump(data, outf)

rgrus.texts.json
lentaruofficial.texts.json
ForbesRussia.texts.json
ru_rbc.texts.json
snob_project.texts.json
GazetaRu.texts.json
lifenews_ru.texts.json
mkomsomolets.texts.json
kpru.texts.json
kommersant.texts.json
meduzaproject.texts.json
fontanka_news.texts.json


In [31]:
data[0]

{'date': '2017-06-24',
 'entities': [('I-ORG', 'Британская'),
  ('I-ORG', 'компании'),
  ('I-LOC', 'Британии'),
  ('I-ORG', 'EDF'),
  ('I-ORG', 'Times'),
  ('I-LOC', 'password'),
  ('I-LOC', 'police1'),
  ('I-ORG', 'The'),
  ('I-ORG', 'Монитор'),
  ('I-PER', 'Тарас'),
  ('I-PER', 'взломанными'),
  ('I-LOC', 'Великобритании'),
  ('I-PER', 'Великобритании'),
  ('I-ORG', 'Центра'),
  ('I-PER', 'Никита'),
  ('I-PER', 'Взломщики'),
  ('I-PER', 'Александр'),
  ('I-PER', 'Взломщики'),
  ('I-ORG', 'Фонтанкой'),
  ('I-ORG', 'MySpace'),
  ('I-PER', 'Приманка'),
  ('I-PER', 'мошенники'),
  ('I-PER', 'Александра'),
  ('I-PER', 'Александр'),
  ('I-LOC', 'США'),
  ('I-PER', 'хакеры'),
  ('I-LOC', 'США'),
  ('I-LOC', 'Украины'),
  ('I-ORG', 'В'),
  ('I-LOC', 'В'),
  ('I-ORG', 'Financial'),
  ('I-LOC', 'Россию'),
  ('I-LOC', '.'),
  ('I-ORG', 'НАТО'),
  ('I-LOC', 'Петербурге')],
 'polarity': -0.043478260869565216,
 'processed_text': ['британск',
  'the',
  'times',
  '24',
  'июн',
  'сообщ',
  'подро

### Train Sentiment Analyzer

In [20]:
with open("train.json", "r") as inpf:
    sa_data = json.load(inpf)
    
for i, entry in enumerate(sa_data):
    sa_data[i]["processed_text"] = word_tokenize_filt(entry["text"], True, True)


In [21]:
def make_word_id_mappings(data):
    counts = defaultdict(int)
    for entry in data:
        for word in entry["processed_text"]:
            counts[word] += 1
    counts = sorted(list(counts.items()), key=lambda x: x[1], reverse=True)
    word_to_id = dict()
    id_to_word = []
    
    for i, (word, _) in enumerate(counts):
        word_to_id[word] = i
        id_to_word.append(word)
    
    return counts, word_to_id, id_to_word
    

In [22]:
_, word_to_id, id_to_word = make_word_id_mappings(sa_data)

In [23]:
import numpy as np
def count_vectorize(words, word_to_id, topK=25000):
    v = np.zeros(topK)
    for w in words:
        idx = word_to_id.get(w, topK - 1)
        if idx >= topK:
            idx = topK - 1
        v[idx] += 1
    return v

In [24]:
X_sa = [count_vectorize(e["processed_text"], word_to_id) for e in sa_data]

In [25]:
X_sa = np.array(X_sa)

In [26]:
import numpy as np

In [27]:
# labels = {
#     "negative": -1,
#     "neutral": 0,
#     "positive": +1
# }

labels = {
    "negative": 0,
    "neutral": 1,
    "positive": 1
}

Y_sa = np.array([labels[e["sentiment"]] for e in sa_data])

In [33]:
import sklearn

In [34]:
from sklearn.model_selection import cross_val_predict

In [35]:
from sklearn import ensemble

In [37]:
rf = ensemble.RandomForestClassifier(n_estimators=2000)

In [ ]:
predicted = cross_val_predict(rf, X_sa, Y_sa, cv=3)

In [188]:
# lr = sklearn.linear_model.LogisticRegression()

In [187]:
sklearn.metrics.confusion_matrix(Y_sa, predicted)

array([[ 596,  770,   68],
       [ 204, 3206,  624],
       [  25, 1086, 1684]])

In [ ]:
counts, word_to_id, id_to_word = make_word_id_mappings(data)

In [ ]:
make_wor

In [146]:
word_tokenize_filt(entry["text"], True, True)

['досудебн',
 'расследован',
 'по',
 'факт',
 'покупк',
 'енпф',
 'пакет',
 'облигац',
 'то',
 '``',
 'бузгул',
 'аурум',
 "'",
 'был',
 'начат',
 'по',
 'инициатив',
 'национальн',
 'банк',
 'рк',
 ',',
 'сообщ',
 'директор',
 'департамент',
 'защит',
 'прав',
 'потребител',
 'и',
 'финансов',
 'услуг',
 'нацбанк',
 'казахста',
 'александр',
 'терент',
 '.',
 '``',
 'основан',
 'для',
 'досудебн',
 'расследован',
 'стал',
 'обращен',
 'национальн',
 'банк',
 ',',
 'письм',
 'от',
 '25',
 'ноябр',
 '2016',
 'год',
 '.',
 'был',
 'обращен',
 'национальн',
 'банк',
 'в',
 'правоохранительн',
 'орга',
 '.',
 'нам',
 'эт',
 'сделк',
 'показа',
 'сомнительн',
 ',',
 'недостаточн',
 'корректн',
 ',',
 'поэт',
 'нацбанк',
 '25',
 'ноябр',
 '2016',
 'год',
 'обрат',
 'в',
 'правоохранительн',
 'орга',
 '.',
 'эт',
 'то',
 ',',
 'что',
 'я',
 'мог',
 'озвуч',
 'на',
 'сегодн',
 '.',
 'идет',
 'следств',
 ',',
 'провод',
 'проверк',
 "'",
 ',',
 '–',
 'сказа',
 'терент',
 '.',
 '28',
 'декабр',


In [144]:
sa_data[8000]

{'id': 10054,
 'sentiment': 'positive',
 'text': 'За 11 месяцев 2015 года объем экспорта обработанной продукции из Казахстана в Китай составил 2,7 млрд. долларов, сообщил председатель правления Казахстанского национального агентства по экспорту и инвестициям Борисбий Жангуразов на пресс-конференции в Астане. По его словам, за 11 месяцев 2015 года объем экспорта обработанной продукции из Казахстана в Китай был на 16% выше, чем за тот же период 2014 года. В настоящее время выявлено более 20 товарных позиций, на которые есть спрос в Китае. Это кондитерские изделия, рис, масложировая продукция, безалкогольные напитки, мясо индейки, а также металлургия и многое другое, поэтому Казахстан может наращивать экспорт этих товаров в Китай, передает ИА «Казах-Зерно». Он также сообщил, что Казахстан экспортирует в Китай продукцию металлургии, химии, урана. Сейчас в мире наблюдается падение цен на основные металлы и продукцию, однако Казахстан имеет неплохие возможности по сохранению темпов экспорта 

In [141]:
pd.Series(s).vlaue

neutral     4034
positive    2795
negative    1434
dtype: int64

In [121]:
data = preprocess_data(data, do_lowercase=True)

В Сумской области Украины задержали двух мужчин, которые на тракторе пытались перевезти в Россию почти три тонны мясной продукции. Об этом сообщает в субботу, 24 июня, украинская государственная пограничная служба. Видео доступно на YouTube-канале ведомства. Трактор с прицепом, груженым мясом, задержали недалеко от границы между украинским населенным пунктом Степок и российским Прилесье. У мужчин не было при себе никаких документов, удостоверяющих личность. Позднее их личности были установлены, задержанные оказались жителями одного из местных населенных пунктов. Они предстанут перед судом за попытку незаконного пересечения границы. Трактор с грузом передадут фискальной службе. В январе гражданин Украины пытался провезти через российско-украинскую границу мясную тушу на санях, но был задержан.


ValueError: This Sequence is Empty

In [11]:
SOURCE_PATTERNS = (
    "по\sсловам\s(?P<source>[\w\.\-\«\»]+)",
    "по\sданным\s(?P<source>[\w\.\-\«\»]+)",
    "по\sсообщению\s(?P<source>[\w\.\-\«\»]+)",
    "рассказал\s(?P<source>[\w\.\-\«\»]+)",
    "рассказывает\s(?P<source>[\w\.\-\«\»]+)",
    "рассказал\s(?P<source>[\w\.\-\«\»]+)\s(?P<publisher>[\w\.\-\«\»]+)",
    "рассказывает\s(?P<source>[\w\.\-\«\»]+)\s(?P<publisher>[\w\.\-\«\»]+)",
    "рассказал\s(?P<publisher>[\w\.\-\«\»]+)\s(?P<source>[\w\.\-\«\»]+)",
    "рассказывает\s(?P<publisher>[\w\.\-\«\»]+)\s(?P<source>[\w\.\-\«\»]+)",
    "сообщил\s(?P<source>[\w\.\-\«\»]+)",
    "сообщает\s(?P<source>[\w\.\-\«\»]+)",
    "сообщил\s(?P<source>[\w\.\-\«\»]+)\s(?P<publisher>[\w\.\-\«\»]+)",
    "сообщает\s(?P<source>[\w\.\-\«\»]+)\s(?P<publisher>[\w\.\-\«\»]+)",
    "сообщил\s(?P<publisher>[\w\.\-\«\»]+)\s(?P<source>[\w\.\-\«\»]+)",
    "сообщает\s(?P<publisher>[\w\.\-\«\»]+)\s(?P<source>[\w\.\-\«\»]+)",
#     "сообщил\s(?P<source>[\w\.\-\«\»]+)\sсо\sссылкой\sна\s(?P<source>[\w\.\-\«\»]+)",
#     "сообщает\s(?P<source>[\w\.\-\«\»]+)\sсо\sссылкой\sна\s(?P<source>[\w\.\-\«\»]+)",
#     "рассказал\s(?P<source>[\w\.\-\«\»]+)\sсо\sссылкой\sна\s(?P<source>[\w\.\-\«\»]+)",
#     "рассказывает\s(?P<source>[\w\.\-\«\»]+)\sсо\sссылкой\sна\s(?P<source>[\w\.\-\«\»]+)",
    "сообщил\s(?P<source>[\w\.\-\«\»]+)\sв\sбеседе\sс\s(?P<publisher>[\w\.\-\«\»]+)",
    "рассказал\s(?P<source>[\w\.\-\«\»]+)\sв\sбеседе\sс\sPubliser",
    "в\s(?P<source>[\w\.\-\«\»]+)\sрассказали\,\sчто",
    "в\s(?P<source>[\w\.\-\«\»]+)\sрассказывают\,\sчто",
    "в\s(?P<source>[\w\.\-\«\»]+)\sсообщили\,\sчто",
    "в\s(?P<source>[\w\.\-\«\»]+)\sотметили\,\sчто",
    "в\s(?P<source>[\w\.\-\«\»]+)\sговорят\,\sчто",
    "в\s(?P<source>[\w\.\-\«\»]+)\sсообщают\,\sчто",
    "в\s(?P<source>[\w\.\-\«\»]+)\sотмечают\,\sчто",
    "в\s(?P<source>[\w\.\-\«\»]+)\sобяснили\,\sчто",
    "в\s(?P<source>[\w\.\-\«\»]+)\sобъясняют\,\sчто",
    "как\sотмечают\sв\s(?P<source>[\w\.\-\«\»]+)\,",
    "как\sотметили\sв\s(?P<source>[\w\.\-\«\»]+)\,",
    "как\sотметил\s(?P<source>[\w\.\-\«\»]+)\,",
    "как\sотмечает\s(?P<source>[\w\.\-\«\»]+)\,",
    "как\sрассказал\s(?P<source>[\w\.\-\«\»]+)\,",
    "как\sрассказывает\s(?P<source>[\w\.\-\«\»]+)\,",
    "подчеркнул\s(?P<source>[\w\.\-\«\»]+)",
    "подчеркивает\s(?P<source>[\w\.\-\«\»]+)",
    "объяснил\s(?P<source>[\w\.\-\«\»]+)",
    "объяснили\sв\s(?P<source>[\w\.\-\«\»]+)",
    "пояснили\sв\s(?P<source>[\w\.\-\«\»]+)",
    "уточнили\sв\s(?P<source>[\w\.\-\«\»]+)",
    "(?P<source>[\w\.\-\«\»]+)\sобъяснил\,\sчто",
    "(?P<source>[\w\.\-\«\»]+)\sподчеркнул\,\sчто",
    "(?P<source>[\w\.\-\«\»]+)\sпояснил\,\sчто",
    "(?P<source>[\w\.\-\«\»]+)\sуточнил\,\sчто",
        )


def get_sources(data):
    sources = []
    for entry in data:
        for sent in sent_tokenize(entry["text"]):
            for pattern in SOURCE_PATTERNS:
                match = re.search(pattern, sent.lower())
                if match:
                    sources.append(match.group("source"))
    return sources

In [12]:
def make_word_id_mappings(data):
    counts = defaultdict(int)
    for entry in data:
        for sent in entry["processed_text"]:
            for word in sent:
                counts[word] += 1
    counts = sorted(list(counts.items()), key=lambda x: x[1], reverse=True)
    word_to_id = dict()
    id_to_word = []
    
    for i, (word, _) in enumerate(counts):
        word_to_id[word] = i
        id_to_word.append(word)
    
    return counts, word_to_id, id_to_word
    
    

In [85]:
counts, word_to_id, id_to_word = make_word_id_mappings(data)